In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

/home/chengxi/.conda/envs/test_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Feature embedding

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

Running on device: cuda:0


In [3]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('train/')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999990
Face detected with probability: 1.000000
Face detected with probability: 0.999880
Face detected with probability: 1.000000
Face detected with probability: 1.000000
Face detected with probability: 0.999995


In [4]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()
embeddings.shape

torch.Size([6, 512])

In [5]:
torch.save(embeddings, 'embeddings.pt')

In [6]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

          Chatur      Dean    Farhan       Joy      Raju    Rancho
Chatur  0.000000  1.096706  1.005982  1.253124  1.331800  1.311198
Dean    1.096706  0.000000  1.149511  1.343693  1.393365  1.386932
Farhan  1.005982  1.149511  0.000000  1.174776  1.288747  1.210451
Joy     1.253124  1.343693  1.174776  0.000000  1.082909  1.194725
Raju    1.331800  1.393365  1.288747  1.082909  0.000000  1.352694
Rancho  1.311198  1.386932  1.210451  1.194725  1.352694  0.000000


In [7]:
embedding_feature = pd.DataFrame(embeddings)
embedding_feature

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,tensor(0.0347),tensor(0.0242),tensor(0.0113),tensor(-0.0110),tensor(-0.0334),tensor(0.0592),tensor(0.0666),tensor(-0.0781),tensor(0.0656),tensor(0.0542),...,tensor(0.0515),tensor(-0.0186),tensor(-0.0244),tensor(-0.0855),tensor(-0.0201),tensor(-0.0033),tensor(0.0445),tensor(0.0218),tensor(-0.0355),tensor(0.0971)
1,tensor(0.0097),tensor(-0.0219),tensor(0.0148),tensor(0.0171),tensor(-0.0354),tensor(0.0314),tensor(-0.0114),tensor(-0.0421),tensor(0.0292),tensor(0.0086),...,tensor(-0.0530),tensor(0.0100),tensor(-0.0142),tensor(-0.0170),tensor(-0.0419),tensor(-0.0250),tensor(-0.0203),tensor(-0.0651),tensor(-0.0349),tensor(0.0235)
2,tensor(-0.0529),tensor(0.0140),tensor(-0.0694),tensor(-0.0185),tensor(-0.0037),tensor(0.0047),tensor(0.0746),tensor(-0.0315),tensor(0.0595),tensor(0.0440),...,tensor(0.0565),tensor(0.0245),tensor(-0.0148),tensor(-0.0425),tensor(-0.0282),tensor(-0.0019),tensor(0.0562),tensor(-0.0208),tensor(-0.0744),tensor(0.0357)
3,tensor(-0.0307),tensor(0.0023),tensor(-0.0885),tensor(-0.0496),tensor(-0.0288),tensor(-0.0226),tensor(0.0637),tensor(0.0389),tensor(-0.0533),tensor(0.0127),...,tensor(0.0711),tensor(-0.0171),tensor(-0.0136),tensor(0.0020),tensor(0.0102),tensor(0.0714),tensor(0.0216),tensor(-0.0883),tensor(-0.0284),tensor(0.0274)
4,tensor(0.0014),tensor(0.0270),tensor(-0.0280),tensor(0.0157),tensor(-0.0276),tensor(0.0820),tensor(0.0263),tensor(0.0308),tensor(-0.1013),tensor(0.0526),...,tensor(0.0726),tensor(-0.0774),tensor(-0.0669),tensor(-0.0025),tensor(-0.0197),tensor(0.0257),tensor(-0.0319),tensor(-0.0278),tensor(-0.0130),tensor(0.0322)
5,tensor(-0.0638),tensor(-0.0488),tensor(-0.0695),tensor(-0.0442),tensor(0.0529),tensor(0.0719),tensor(0.0591),tensor(0.0174),tensor(0.0107),tensor(0.0643),...,tensor(0.0010),tensor(-0.0012),tensor(0.0351),tensor(-0.0877),tensor(-0.0374),tensor(0.0178),tensor(0.0683),tensor(-0.0014),tensor(-0.0956),tensor(0.0124)


# Test new image

In [8]:
from PIL import Image
import glob

In [9]:
test_list = os.listdir('test/')
test_list = sorted(test_list)
test_list

['Chatur', 'Dean', 'Farhan', 'Joy', 'Raju', 'Rancho']

In [10]:
accuracy = {}
for test_file in test_list:
    test_results = []
    img_files = glob.glob('test/'+test_file+'/*.png')
    for img_file in img_files:
        img = Image.open(img_file)
        img_cropped = mtcnn(img)
        if img_cropped is not None:
            img_cropped = img_cropped.to(device) # transfor to cuda
        else:
            break
        img_embedding = resnet(img_cropped.unsqueeze(0)).detach().cpu() # transfor to cpu
        dists = [(img_embedding - e1).norm().item() for e1 in embeddings]
        test_results.append(dists.index(min(dists))+1)
        
    accuracy[test_file] = test_results

In [11]:
accuracy

{'Chatur': [1],
 'Dean': [2, 2],
 'Farhan': [3],
 'Joy': [4, 4],
 'Raju': [5],
 'Rancho': [6, 6]}

In [12]:
k = list(accuracy.keys())
v = list(accuracy.values())
df = pd.DataFrame(list(zip(k,v)),columns=['target','accuracy'])
df.to_csv('accuracy.csv')